# Get started with the Estimator primitive

Learn how to set up and use the Estimator primitive program.


## Overview

The Estimator primitive lets you efficiently calculate and interpret expectation values of quantum operators required for many algorithms. You can specify a list of circuits and observables, then evaluate expectation values and variances for a given parameter input.  


## Prepare the environment

1. Follow the steps in the [getting started guide](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/getting_started.html) to get your quantum service instance ready to use.

2. You'll need at least one circuit to submit to the program. Our examples all have circuits in them, but if you want to submit your own circuit, you can use Qiskit to create one. To learn how to create circuits by using Qiskit, see the [Circuit basics tutorial](https://qiskit.org/documentation/tutorials/circuits/01_circuit_basics.html).

3. Create a list of observables. Observables let you define the properties of the circuit that are relevant to your problem and enable you to efficiently measure their expectation value. For simplicity, you can use the [PauliSumOp class](https://qiskit.org/documentation/stubs/qiskit.opflow.primitive_ops.html#module-qiskit.opflow.primitive_ops) in Qiskit to define them, as illustrated in the example below.


## Start a session

With Qiskit Runtime primitives, we introduce the concept of a session or a factory that allows you to define a job as a collection of iterative calls to the quantum computer. When you start a session, it caches the data you send so it doesn't have to be transmitted to the Quantum Datacenter on each iteration.


### Specify program inputs

The Estimator.run() method takes in the following arguments:

- **circuits**: a list of (parameterized) circuits that you want to investigate.
- **observables**: a list of observables to measure the expectation values.
- **parameter_values**: an optional list of concrete parameters to be bound.
- **skip_transpilation**: circuit transpilation is skipped if set to `True`. Default value is `False`.
- **service**: the `QiskitRuntimeService` instance to run the program on. If not specified, the default saved account for `QiskitRuntimeService` is initialized.
- **run_options**: A collection of kwargs passed to `backend.run()`.
    - **shots**: Number of repetitions of each circuit, for sampling.
    - **qubit_lo_freq**: List of default qubit LO frequencies in Hz.
    - **meas_lo_freq**: List of default measurement LO frequencies in Hz.
    - **schedule_los**: Experiment LO configurations, frequencies are given in Hz.
    - **rep_delay**: Delay between programs in seconds. Only supported on certain
        backends (if ``backend.configuration().dynamic_reprate_enabled=True``).
    - **init_qubits**: Whether to reset the qubits to the ground state for each shot.
    - **use_measure_esp**: Whether to use excited state promoted (ESP) readout for measurements
        which are the terminal instruction to a qubit. ESP readout can offer higher fidelity
        than standard measurement sequences.

You can find more details in [the Estimator API reference](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Estimator.html).

Example:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

service = QiskitRuntimeService()

psi1 = RealAmplitudes(num_qubits=2, reps=2)
psi2 = RealAmplitudes(num_qubits=2, reps=3)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

## Write to & read from a session

Running a job and returning the results are done by writing to and reading from the session. After the results are returned, the session is automatically closed.


### Run the job & print results

Run the job, specifying your previously defined inputs and options.  Use `circuits`, `observables`, and `parameter_values` to use a specific parameter and observable with the specified circuit.

For example, this line `psi1_H23_result = estimator(circuits=[0, 0], observables=[1, 2], parameter_values=[theta1]*2)` specifies the following:

- Return the value for using observable `H2` and parameter `theta1` with the circuit `psi1`.
- Return the value for using observable `H3` and parameter `theta1` with the circuit `psi1`.

You can also pass circuits and observables as objects.
For example, `psi1_H23_result = estimator(circuits=[ps1, ps1], observables=[H2, H3], parameter_values=[theta1]*2)`

In [5]:
with Session(service) as session:
    estimator = session.estimator()
    estimator.options.backend = 'ibmq_qasm_simulator'
    estimator.settings.transpilation.optimization_level = 1

    theta1 = [0, 1, 1, 2, 3, 5]
    theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
    theta3 = [1, 2, 3, 4, 5, 6]

    # calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
    psi1_H1 = estimator.run(circuits=[psi1], observables=[H1], parameter_values=[theta1])
    print(psi1_H1.result())

    # calculate [ <psi1(theta1)|H2|psi1(theta1)>, <psi1(theta1)|H3|psi1(theta1)> ]
    psi1_H23 = estimator.run(circuits=[psi1, psi1], observables=[H2, H3], parameter_values=[theta1]*2)
    print(psi1_H23.result())

    # calculate [ <psi2(theta2)|H2|psi2(theta2)> ]
    psi2_H2 = estimator.run(circuits=[psi2], observables=[H2], parameter_values=[theta2])
    print(psi2_H2.result())

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>, <psi1(theta3)|H1|psi1(theta3)> ]
    psi1_H1_job = estimator.run(circuits=[psi1, psi1], observables=[H1, H1], parameter_values=[theta1, theta3])
    print(psi1_H1_job.result())

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>,
    #             <psi2(theta2)|H2|psi2(theta2)>,
    #             <psi1(theta3)|H3|psi1(theta3)> ]
    psi12_H23 = estimator.run(circuits=[psi1, psi2, psi1], observables=[H1, H2, H3], parameter_values=[theta1, theta2, theta3])
    print(psi12_H23.result())

EstimatorResult(values=array([1.53710938]), metadata=[{'variance': 9.249675750732422, 'shots': 1024}])
EstimatorResult(values=array([-0.54492188,  0.06835938]), metadata=[{'variance': 0.7030601501464844, 'shots': 1024}, {'variance': 1.9969749450683594, 'shots': 1024}])
EstimatorResult(values=array([0.12890625]), metadata=[{'variance': 0.9833831787109375, 'shots': 1024}])
EstimatorResult(values=array([1.56445312, 1.15234375]), metadata=[{'variance': 8.791683197021484, 'shots': 1024}, {'variance': 12.287948608398438, 'shots': 1024}])
EstimatorResult(values=array([ 1.50976562,  0.1171875 , -1.09179688]), metadata=[{'variance': 9.621807098388672, 'shots': 1024}, {'variance': 0.98626708984375, 'shots': 1024}, {'variance': 1.2446861267089844, 'shots': 1024}])


The results align with the parameter - circuit - observable tuples specified previously.  For example, the first result: `EstimatorResult(values=array([1.53710938]), metadata=[{'variance': 9.249675750732422, 'shots': 1024}])` is the output of the parameter labeled `theta1` and observable `H1` being sent to the first circuit.